In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [3]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [4]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [6]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([92122, 77864, 11847, 30873])
inputs : 
tensor([[28, 68, 58, 72,  1, 73, 61, 58],
        [ 1, 65, 62, 60, 61, 73, 58, 71],
        [28, 68, 71, 68, 73, 61, 78, 11],
        [ 1, 71, 58, 54, 71, 11,  1, 43]], device='cuda:0')
targets : 
tensor([[68, 58, 72,  1, 73, 61, 58,  1],
        [65, 62, 60, 61, 73, 58, 71,  1],
        [68, 71, 68, 73, 61, 78, 11,  3],
        [71, 58, 54, 71, 11,  1, 43, 65]], device='cuda:0')


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train' , 'val' ]:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out        

### Making the Bigram Language model

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


h"xaS0?ZqjSqC&X**ZfF;.Vv[WOH]f:i"x9gpP_SYyFtM5O*([n2vtRvw]CPMM2,,sv1xsK*7)o&pC1!I6Fti2wFReUD]lxH﻿ stRiw7)r5e﻿OpemdAb!O*URBPz-i"Z0jUljZ;&QC"k5,VzjXiwFX*R-jX-X*d7FTMJvJdI[C7I1
wh8:"P"wvXryM 1yB6gluIed4mlAgssB"&3
x_1?HCb3RyCV5Mydw?73o8S4:d8MM.bwIDm8FNM 1v3("&CKyY6LdIryMn*﻿Bk0yxuK_BHxRaPc3EwQvNidVz﻿uoQ﻿lch7Y,C?IXa.k-5ODgfea_vck zg'5!-YTXTufgD7*Pz5K?Ir9nZpoQn"[FHxag-peUWH3;;mHw?H52L9GbY7pFNJ]fg162&FjV'﻿(MW[﻿)S6BHOL5LRTS5XVoQeYM.o)Ej0H3" .UgOms"4Y0us6vp P*RBax[k&k
:irI!ho;.!x[b3!6iw?ELZpnyi"u'mRvc7tyc


### Optimizer

In [15]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}. train loss {losses['train']},  val loss: {losses['val']}")
    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([131771,  49912,   9071, 172635])
tensor([133472, 129268,  36942, 180292])
tensor([ 31007, 181166,  29721,  92788])
tensor([ 13899,  46765, 126348, 160493])
tensor([163323,  57978, 125538,  60006])
tensor([119201, 107496, 116135, 110238])
tensor([149829, 181381, 153671,  88606])
tensor([166770,  99445, 182347, 118979])
tensor([112230,  42006,  85446,  33804])
tensor([185796, 161125, 154026, 122905])
tensor([ 13621,  93513,  44723, 183646])
tensor([175243,  96663,  14156,  16429])
tensor([ 56279,  73063,  27194, 159909])
tensor([ 93648,  64411, 135847, 111390])
tensor([ 81068, 103700, 156260, 101175])
tensor([143332,  28840, 156614,  15545])
tensor([166457, 144595, 153317, 164878])
tensor([133086, 136800,  79193,  68529])
tensor([119873, 100984,  60342,  19430])
tensor([ 50491,  53810,  44560, 166867])
tensor([ 66329,  21367, 147412,  87943])
tensor([155822,  59119,  18573,  73621])
tensor([ 66690, 125491, 140246,  28996])
tensor([97505, 23037, 88223, 20304])
tensor([ 79090,   35

In [16]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


som'ledisiz. thes theane h thtint wnd on t musherey "IR. ed wind wititather drourdd.
Dowand L.
tilint-g'mpaputh u hes whansprutr arcte.  d s s
lds oug d r-aibs f anend an "W24LIzz.
garixhad  hizled Jis t oe touthanc61jy livere mur
"N4!qLked hetoustone ano, has Vo g Ty tonen."Wewhero t

acad had aleal tre aritly, cre riars and ped w Thand oveeanok, veath.
Gale onditury u ateecan beke d ats.

T." thelun

g. wie ou. L5! p ce, floy.
-d the
angret s asaleref thethe O!Pr. heanknthanomainy't t che-ac."
